Part 1</br>
- Write a map reduce program that will examine this file, then print, for each column, the name of the column and the largest value found in that column, and for each row, the name of the row and the smallest value found in that row.  Note that the column and row names are strings, a single character.  The output should be of the form:
  - "A", 564
  - "B", 329
- There should be 20 lines in the report, and they do not have to be in any particular order.

Part 2
- This is a second map reduce program that is similar to the first part, but has an additional feature.  For a column, the output should show the column name, the largest value in that column, and the name of one of the rows where this maximum value is found (there may be more than one cell with the same maximum value).  For each row, the output should show the row name, the smallest value in that row, and the name of one of the columns where this minimum value is found.
- Again, the output should have 20 lines, and they do not have to be in any order.  An example of one line of the report is:
  - "A", {"value":564, "example":"M"}

Part 3</br>
- This is a third map reduce program that is a further extension of Part 2.  In this case, for the output, we want to include a list of all rows/columns that have that maximum/minimum value.  Note that most rows and columns will have a unique extreme, but some will have duplicates.  This output might look like this:
  - "A", ["value":564, "examples": ["M"]}
  - "D", ["value":437, "examples": ["L",  "P"]}